In [3]:
import pandas as pd
import numpy as np
from random import sample
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import pickle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trainwithzerostopredict.csv')
topredict= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/topredict.csv')
df_withoutzero= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trainwithoutzeros.csv')

***df has all the topredict values entered as 0 for the Book-Rating column.***</br>
***topredict has all the values that we want to predict.***</br>
***df_withoutzero has all the topredict all the values including the values in the test dataset.***



In [6]:
df['User-ID'].nunique()

77805

In [7]:
topredict['User-ID'].nunique()

19935

##**1000 users that rate the most**

In [8]:
df_num_rating= df.groupby('User-ID').agg(Number_of_ratings=('Book-Rating','count')).reset_index()
# Num ratings count

In [9]:
df_num_rating.sort_values(by=['Number_of_ratings'], ascending=False, inplace=True)
df_num_rating

,User-ID,Number_of_ratings
3160,11676,8524
27626,98391,5802
43027,153662,1969
52924,189835,1906
6510,23902,1395
...,...,...
23601,84129,1
46266,165812,1
9606,34231,1
46268,165826,1


In [10]:
top_users= df_num_rating['User-ID'][:1000]
# top 1500 users

##**To Predict User List and Batch Designation**

In [11]:
topredict_users= topredict['User-ID'].unique()
topredict_users

array([    17,     56,    114, ..., 278844, 278851, 278854])

In [12]:
len(topredict_users)

19935

**We divide the test into 20 folds because of memory limitations. We will predict the test values by loading in batches**

In [13]:
dict_batch= {}
for i in range(0,len(topredict_users),500):
  dict_batch['fold_{}'.format(i//500)]= topredict_users[i:i+500]

In [14]:
dict_batch #fold_0 to fold_19#

{'fold_0': array([  17,   56,  114,  160,  183,  242,  243,  254,  272,  289,  300,
         362,  383,  388,  392,  408,  424,  440,  441,  444,  446,  472,
         476,  487,  503,  505,  507,  566,  619,  625,  626,  638,  640,
         643,  651,  657,  695,  709,  726,  741,  744,  746,  753,  776,
         786,  805,  819,  850,  853,  882,  885,  896,  899,  900,  901,
         914,  929, 1009, 1021, 1025, 1031, 1063, 1075, 1083, 1096, 1113,
        1116, 1129, 1131, 1155, 1161, 1167, 1178, 1184, 1211, 1235, 1248,
        1249, 1254, 1261, 1297, 1343, 1376, 1409, 1412, 1421, 1424, 1435,
        1436, 1467, 1485, 1517, 1548, 1558, 1570, 1585, 1596, 1597, 1619,
        1652, 1660, 1667, 1674, 1688, 1696, 1733, 1790, 1791, 1797, 1805,
        1830, 1848, 1903, 1990, 2009, 2010, 2024, 2030, 2033, 2041, 2046,
        2084, 2090, 2103, 2106, 2110, 2132, 2134, 2179, 2189, 2197, 2203,
        2240, 2255, 2276, 2287, 2288, 2313, 2333, 2337, 2354, 2358, 2363,
        2385, 2399, 2404, 24

In [15]:
# this is the matrix factorization function from the MF notebook. 
def MF(M,k,max_it,lambd,mu):
    n=M.size()[0]
    m= M.size()[1]
    nonzero=len(M.nonzero())
    index= M.nonzero().split(1, dim=1)
    #param=torch.rand(n*k+k*m,dtype=float,requires_grad=True)
    param1=torch.rand((n,k),dtype=torch.float,requires_grad=True)
    param2=torch.rand((k,m),dtype=torch.float,requires_grad=True)

    opt1= torch.optim.Adam([param1],lr=0.1)
    opt2= torch.optim.Adam([param2],lr=0.1)
    
    #scheduler1= ReduceLROnPlateau(opt1, 'min') 
    #scheduler2 = ReduceLROnPlateau(opt2, 'min')
    

    #def get_loss(params,params_hat):
        #return torch.sum(torch.square(params- params_hat))

    def run_iterations(max_it):
        loss_record=[]
        converged=False
        for it in tqdm(range(max_it)):
            if it%2==0:
                opt1.zero_grad(set_to_none=True)
                #torch.matmul(param[:n*k].reshape(n,k), pam[n*k:].reshape(k,m))
                loss=torch.sum(torch.square(torch.matmul(param1, param2)[index]- M[index])) + lambd*torch.sum(torch.square(param1))+mu*torch.sum(torch.square(param2))
                loss_record.append(loss.item())
                loss.backward()
                opt1.step()
                #scheduler1.step(loss)
            else:
                opt2.zero_grad(set_to_none=True)
                #torch.matmul(param[:n*k].reshape(n,k), pam[n*k:].reshape(k,m))
                loss=torch.sum(torch.square(torch.matmul(param1, param2)[index]- M[index])) + lambd*torch.sum(torch.square(param1))+mu*torch.sum(torch.square(param2))
                loss_record.append(loss.item())
                loss.backward()
                opt2.step()
                #scheduler2.step(loss)
        display("loss record: {}".format(loss_record[-1]))
        return torch.matmul(param1,param2)
    return run_iterations(max_it)

**The following loop loads the ith batch of the train dataset and constructs a matrix using the ith batch and the subset of the train dataset containing the 1000 top users. Then it computes the matrix factorization prediction on the ith batch. It calculates the MSE for each batch in each iteration.**

In [ ]:
mse_batch=[] # this will collect the batch MSE
indices_batch=[] # this will collect the indices of the test dataset in each batch
for i in range(len(dict_batch)): #length of dict_batch is 20 because we have 20 folds
  df_batch= topredict[topredict['User-ID'].isin(dict_batch['fold_{}'.format(i)])].reset_index(drop=True)
  df_matrix= df[df['User-ID'].isin(list(dict_batch['fold_{}'.format(i)])+list(top_users))].reset_index(drop=True) #combines data from 500 top users and the test batch
  mat= df_matrix.pivot(index='User-ID',columns='ISBN',values='Book-Rating') .fillna(0) # matrix using pivot
  matrix= torch.tensor(mat.values)/10 #converts to tensor and scales the values by 1/10
  dict_user= dict(zip(sorted(set(df_matrix['User-ID'])),range(len(sorted(set(df_matrix['User-ID']))))))
  dict_book= dict(zip(sorted(set(df_matrix['ISBN'])),range(len(sorted(set(df_matrix['ISBN']))))))
  index1=[] #this will collect the row index
  index2=[] #this will collect the column index
  for j in range(len(df_batch)):
    index1.append([dict_user[df_batch['User-ID'][j]]])
    index2.append([dict_book[df_batch['ISBN'][j]]])
  indices_topred = (torch.tensor(index1),torch.tensor(index1)) #this contains the indices from the matrix whose values we want to predict
  # this is a bit peculiar but torch tensors work this way
  df_actual= df_withoutzero[df_withoutzero['User-ID'].isin(list(dict_batch['fold_{}'.format(i)])+list(top_users))]
  # df_actual is the matrix with the ratings of the test dataset included
  # we need this to calculate MSE
  actual_matrix= df_actual.pivot(index='User-ID',columns='ISBN',values='Book-Rating') .fillna(0)
  actual_matrix= torch.tensor(actual_matrix.values)/10 # convert to tensor and scale like above

  mse_batch.append(100*torch.sum(torch.square(MF(matrix,10,1000,0.1,0.1)[indices_topred]- actual_matrix[indices_topred])))
  # we multiply by 10^2 because we scaled by 1/10 and squared
  # the values 25,0.1,0.1 come from the MF validation notebook where we performed validation for parameter tuning 
  indices_batch.append(len(indices_topred[0]))
  display("iteration: {}".format(i))
  display("rmse_batch: {}".format(torch.sqrt(mse_batch[-1]/indices_batch[-1])))
 






100%|██████████| 1000/1000 [08:52<00:00,  1.88it/s]


'loss record: 1610.5978465752178'

'iteration: 0'

'rmse_batch: 2.1254843011057507'

100%|██████████| 1000/1000 [09:24<00:00,  1.77it/s]


...

100%|██████████| 1000/1000 [09:23<00:00,  1.78it/s]


'loss record: 1589.1649763898017'

'iteration: 36'

'rmse_batch: 2.522839723983094'

  4%|▎         | 37/1000 [00:21<09:12,  1.74it/s]

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/mse_batch", "wb") as fp:   #Pickling
  pickle.dump(mse_batch, fp)

with open("/content/drive/MyDrive/Colab Notebooks/indices_num_batch", "wb") as fp:   #Pickling
  pickle.dump(indices_batch, fp)

In [ ]:
mse= sum(mse_batch)/sum(indices_batch)

In [ ]:
mse

In [ ]:
rmse= np.sqrt(mse.detach())
rmse